In [2]:
import pandas as pd

import json

In [4]:
df = pd.read_csv('data/covid_19_indonesia_time_series_all.csv')
df.head()

,Date,Location ISO Code,Location,New Cases,New Deaths,New Recovered,New Active Cases,Total Cases,Total Deaths,Total Recovered,...,Longitude,Latitude,New Cases per Million,Total Cases per Million,New Deaths per Million,Total Deaths per Million,Case Fatality Rate,Case Recovered Rate,Growth Factor of New Cases,Growth Factor of New Deaths
0,3/1/2020,ID-JK,DKI Jakarta,2,0,0,2,489,20,73,...,106.836118,-6.204699,0.18,45.09,0.0,1.84,4.09%,14.93%,NaN,NaN
1,3/2/2020,ID-JK,DKI Jakarta,2,0,0,2,491,20,73,...,106.836118,-6.204699,0.18,45.27,0.0,1.84,4.07%,14.87%,1.0,1.0
2,3/2/2020,IDN,Indonesia,2,0,0,2,2,0,0,...,113.921327,-0.789275,0.01,0.01,0.0,0.00,0.00%,0.00%,NaN,NaN
3,3/2/2020,ID-JB,Jawa Barat,3,0,0,3,12,5,14,...,107.603708,-6.920432,0.07,0.27,0.0,0.11,41.67%,116.67%,NaN,NaN
4,3/2/2020,ID-RI,Riau,1,0,0,1,2,1,2,...,101.805109,0.511648,0.16,0.33,0.0,0.16,50.00%,100.00%,NaN,NaN


In [5]:
df['cases_per_100k'] = df['Total Cases per Million'].apply(lambda x: x/100)

In [7]:
# adjust names from json file
## adjust names in df to match names in indo_states_map
df = df.replace('Bangka Belitung', 'Kepulauan Bangka Belitung')
df = df.replace("DKI Jakarta", 'Dki Jakarta')


array(['Dki Jakarta', 'Indonesia', 'Jawa Barat', 'Riau', 'Banten',
       'Sulawesi Tenggara', 'Kalimantan Timur',
       'Daerah Istimewa Yogyakarta', 'Sumatera Utara', 'Jawa Tengah',
       'Jawa Timur', 'Sulawesi Selatan', 'Maluku', 'Papua',
       'Maluku Utara', 'Aceh', 'Kalimantan Tengah', 'Lampung',
       'Sulawesi Tengah', 'Sumatera Barat', 'Papua Barat',
       'Kalimantan Utara', 'Sulawesi Barat', 'Sumatera Selatan', 'Bali',
       'Kalimantan Barat', 'Jambi', 'Kalimantan Selatan',
       'Kepulauan Bangka Belitung', 'Kepulauan Riau', 'Bengkulu',
       'Nusa Tenggara Barat', 'Sulawesi Utara', 'Nusa Tenggara Timur',
       'Gorontalo'], dtype=object)

In [9]:

# 2. add id column with feature.id from geojson
indo_geo = json.load(open('data/indo_level1_id.geojson'))


{'Shape_Leng': 27.3673635506,
 'Shape_Area': 4.62543727674,
 'ADM1_EN': 'Aceh',
 'ADM1_PCODE': 'ID11',
 'ADM1_REF': None,
 'ADM1ALT1EN': None,
 'ADM1ALT2EN': None,
 'ADM0_EN': 'Indonesia',
 'ADM0_PCODE': 'ID',
 'date': '2019-12-20',
 'validOn': '2020-04-01',
 'validTo': None}

In [16]:

provi_id_map = {}

for province in indo_geo['features']:
    provi_id_map[province['properties']['ADM1_EN']] = province['id']
df['id'] = df['Location'].apply(lambda x: provi_id_map[x] if x in provi_id_map else 0)
df.head()

,Date,Location ISO Code,Location,New Cases,New Deaths,New Recovered,New Active Cases,Total Cases,Total Deaths,Total Recovered,...,New Cases per Million,Total Cases per Million,New Deaths per Million,Total Deaths per Million,Case Fatality Rate,Case Recovered Rate,Growth Factor of New Cases,Growth Factor of New Deaths,cases_per_100k,id
0,3/1/2020,ID-JK,Dki Jakarta,2,0,0,2,489,20,73,...,0.18,45.09,0.0,1.84,4.09%,14.93%,NaN,NaN,0.4509,6
1,3/2/2020,ID-JK,Dki Jakarta,2,0,0,2,491,20,73,...,0.18,45.27,0.0,1.84,4.07%,14.87%,1.0,1.0,0.4527,6
2,3/2/2020,IDN,Indonesia,2,0,0,2,2,0,0,...,0.01,0.01,0.0,0.00,0.00%,0.00%,NaN,NaN,0.0001,0
3,3/2/2020,ID-JB,Jawa Barat,3,0,0,3,12,5,14,...,0.07,0.27,0.0,0.11,41.67%,116.67%,NaN,NaN,0.0027,9
4,3/2/2020,ID-RI,Riau,1,0,0,1,2,1,2,...,0.16,0.33,0.0,0.16,50.00%,100.00%,NaN,NaN,0.0033,26


In [17]:
df.to_csv('data/indo_province_cases.csv')